 
#<font color='yellow'>***/***</font> ***ProLink*** <font color='yellow'>***/***</font>  

2022 @ *Universidad de Zaragoza*

For more information please visit https://github.com/unizar-flav/ProLink




In [ ]:
#@title Install environment
#@markdown Run this cell first and only once! It will install all the dependencies required to run the ProLink package.
%%capture

import os

! apt-get update
! wget https://www.megasoftware.net/do_force_download/megax-cc_10.2.6-1_amd64.deb
! apt-get install -y ghostscript mash muscle ./megax-cc_10.2.6-1_amd64.deb

! wget https://github.com/soedinglab/MMseqs2/releases/download/14-7e284/mmseqs-linux-avx2.tar.gz
! tar -xzf mmseqs-linux-avx2.tar.gz
os.environ['PATH'] += ":/content/mmseqs/bin"

! git clone https://github.com/unizar-flav/ALFATClust.git
os.environ['PATH'] += ":/content/ALFATClust/main"

%pip install biopython python-igraph leidenalg weblogo
%pip install git+https://github.com/unizar-flav/ProLink.git

from datetime import datetime
from google.colab import files
from ProLink import pro_link

In [ ]:
#@title Introduce the parameters { run: "auto" }

parameters = dict()

#@markdown ----QUERY PROTEINS UNIPROT CODES----
query_proteins = "ABQ62066.1, ABQ62091.1, ABQ62490.1" #@param {type:"string"}

#@markdown ----BLAST PARAMETERS----
hitlist_range = 5000 #@param {type:"integer"}
parameters['hitlist_range'] = hitlist_range

blast_database = "Reference Proteins (refseq_protein)" #@param ["Non redundant protein sequences (nr)", "Reference Proteins (refseq_protein)", "Model organisms (landmark)", "Protein Data Bank proteins (pdb)"]
parameters['blast_database'] = blast_database.split('(')[1].split(')')[0] # database name between parenthesis

pro_blast_ = False #@param {type:"boolean"}
parameters['pro_blast_'] = pro_blast_

#@markdown ----CLUSTERING----
cluster_seqs = True #@param {type:"boolean"}
parameters['cluster_seqs'] = cluster_seqs

similarity = 0.5 #@param {type:"number"} #Initial similarity treshold to group the sequences into clusters.
parameters['similarity'] = similarity

pro_clustering_ = True #@param {type:"boolean"}
parameters['pro_clustering_'] = pro_clustering_

#@markdown ----PFAM DOMAINS----
check_pfam_domains = True #@param {type:"boolean"}
parameters['check_pfam_domains'] = check_pfam_domains

#@markdown ----ALIGNMENT----
align_seqs = True #@param {type:"boolean"}
parameters['align_seqs'] = align_seqs

#@markdown ----SEQUENCE LOGO GENERATION----
generate_logo = True #@param {type:"boolean"}
parameters['generate_logo'] = generate_logo

#@markdown ----PHYLOGENETIC TREE GENERATION----
generate_tree = True #@param {type:"boolean"}
parameters['generate_tree'] = generate_tree

tree_type = "NJ" #@param ["NJ", "ML"]
parameters['tree_type'] = tree_type

boostrap_replications = "250" #@param ["100", "250", "500", "1000", "2000", "5000"]
parameters['boostrap_replications'] = boostrap_replications

#@markdown For more advanced options, please feel free to edit the ProLink/parameters.cfg file.

In [ ]:
#@title Execute the script
#@markdown Running the clustering (This may take a while)

now = str(datetime.now()).replace(" ", "_")
outputs_dir = f"outputs_{now}"
up = {'outputs_dir': outputs_dir}
parameters['Outputs'].update(up)
print(now)
print(parameters)
download_outputs = True #@param {type:"boolean"}

pro_link(query_proteins, **parameters) 

if download_outputs:
  downloadfile = f"/content/{outputs_dir}"
  output_filename = f"ProLink_{outputs_dir}.zip"
  os.system(f"zip -r {output_filename} {downloadfile}")
  files.download( output_filename )

In [ ]:
#@title (Optional)  Download outputs manually { display-mode: "form" }
#@markdown Copy the path of the file/directory below and run the cell to download.
downloadfile = "" #@param {type: "string"}

output_filename = 'ProLink_outputs.zip'
os.system(f"zip -r {output_filename} {downloadfile}")
files.download( output_filename )